In [1]:
import pymongo
import pandas as pd
import re

In [2]:
file='C:/Users/psk/3rd_backup/Kream_detail.csv'

In [3]:
# unrename:01 없애기 옵션 : index_col=0
df = pd.read_csv(file,index_col=0)
df

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,"200,000",553558-053,-,GREY/WHITE/CREAM,"129,000원","202,000","201,000",https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,"1,555,000",DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,"189,000원","1,540,000","2,230,000",https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,"702,000",555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","600,000","818,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,"312,000",DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,"249,000원","299,000","313,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,"244,000",555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,"209,000원","207,000","270,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2988,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),-,CD7071-001,19/05/25,BLACK/BLACK-WHITE-VARSITY RED,"$180 (약 234,600원)",-,-,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
2989,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,-,810551-050,15/06/14,BLACK/DARK GREY-WHITE-CHROME,"$250 (약 325,200원)",-,-,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
2990,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,"290,000",852625-650,16/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,"$220 (약 286,200원)","340,000","180,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
2991,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,"350,000",AH7233-105,18/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,"$180 (약 234,100원)","380,000","150,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2993 entries, 0 to 2992
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   _id                 2993 non-null   object
 1   brand               2993 non-null   object
 2   prod_kr_name        2993 non-null   object
 3   rescent_price       2993 non-null   object
 4   model_no            2993 non-null   object
 5   release_date        2993 non-null   object
 6   colors              2993 non-null   object
 7   release_price       2993 non-null   object
 8   instant_buy_price   2993 non-null   object
 9   instant_sell_price  2993 non-null   object
 10  img_url             2993 non-null   object
dtypes: object(11)
memory usage: 280.6+ KB


In [5]:
df['release_price']

0                 129,000원 
1                 189,000원 
2                 199,000원 
3                 249,000원 
4                 209,000원 
               ...         
2988     $180 (약 234,600원) 
2989     $250 (약 325,200원) 
2990     $220 (약 286,200원) 
2991     $180 (약 234,100원) 
2992     $110 (약 143,100원) 
Name: release_price, Length: 2993, dtype: object

In [6]:
# price 전처리 
# ex) 1,540,000 -> 1540000
def price_pro(data):
    data = data.replace('-','0')
    data= data.replace(',','')
    return int(data)

In [7]:
# data1= '$180 (약 234,600원)'

In [8]:
# re.findall('\d[0-9,]+원',data1)

In [9]:
# re.findall('\d[0-9]*원',data1)[0].replace('원','').replace(',','')

In [10]:
# 출시가격 전처리
# ex) $110 (약 143,100원) -> 143100
def release_price_pro(data):
    filtered=re.findall('\d[0-9,]*원',data)
    if filtered:
        data = filtered[0].replace('원','').replace(',','')
    else:
        data='0'
        
    return int(data)

    

In [11]:
df2=df.copy()
df2.head()

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,"200,000",553558-053,-,GREY/WHITE/CREAM,"129,000원","202,000","201,000",https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,"1,555,000",DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,"189,000원","1,540,000","2,230,000",https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,"702,000",555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","600,000","818,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,"312,000",DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,"249,000원","299,000","313,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,"244,000",555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,"209,000원","207,000","270,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...


In [12]:
# 전체 데이터 적용
df2['rescent_price'] = df2['rescent_price'].apply(price_pro)
df2['instant_buy_price'] = df2['instant_buy_price'].apply(price_pro)
df2['instant_sell_price'] = df2['instant_sell_price'].apply(price_pro)
df2

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,200000,553558-053,-,GREY/WHITE/CREAM,"129,000원",202000,201000,https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,1555000,DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,"189,000원",1540000,2230000,https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,702000,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원",600000,818000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,312000,DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,"249,000원",299000,313000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,244000,555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,"209,000원",207000,270000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2988,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),0,CD7071-001,19/05/25,BLACK/BLACK-WHITE-VARSITY RED,"$180 (약 234,600원)",0,0,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
2989,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,0,810551-050,15/06/14,BLACK/DARK GREY-WHITE-CHROME,"$250 (약 325,200원)",0,0,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
2990,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,290000,852625-650,16/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,"$220 (약 286,200원)",340000,180000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
2991,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,350000,AH7233-105,18/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,"$180 (약 234,100원)",380000,150000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [13]:
df2['release_price'] = df2['release_price'].apply(release_price_pro)
df2

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,200000,553558-053,-,GREY/WHITE/CREAM,129000,202000,201000,https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,1555000,DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,189000,1540000,2230000,https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,702000,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,199000,600000,818000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,312000,DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,249000,299000,313000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,244000,555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,209000,207000,270000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2988,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),0,CD7071-001,19/05/25,BLACK/BLACK-WHITE-VARSITY RED,234600,0,0,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
2989,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,0,810551-050,15/06/14,BLACK/DARK GREY-WHITE-CHROME,325200,0,0,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
2990,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,290000,852625-650,16/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,286200,340000,180000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
2991,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,350000,AH7233-105,18/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,234100,380000,150000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [14]:
# data1='20/11/13'

In [15]:
# data1[:2]

In [16]:
# data1 = '20'+ data1
# # data1

In [17]:
# 출시일 전처리
# 14/12/20 -> 2014/12/20
def date_preprocessing(data):
    data = data.strip()
    if data != '-':
        if int(data[:2]) < 50:
            data = '20' + data
        elif int(data[:2]) >= 50:
            data = '19' + data
    
    data = data.replace('-', '1900/01/01')

    return data

In [18]:
df2['release_date'] = df2['release_date'].apply(date_preprocessing)
df2

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,200000,553558-053,1900/01/01,GREY/WHITE/CREAM,129000,202000,201000,https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,1555000,DM7866-140,2021/08/13,SAIL/BLACK/MILITARY BLUE,189000,1540000,2230000,https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,702000,555088-105,2020/11/13,SAIL/DARK MOCHA-BLACK-BLACK,199000,600000,818000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,312000,DH6927-111,2022/05/23,WHITE/BLACK/NEUTRAL GREY,249000,299000,313000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,244000,555088-108,2022/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,209000,207000,270000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2988,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),0,CD7071-001,2019/05/25,BLACK/BLACK-WHITE-VARSITY RED,234600,0,0,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
2989,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,0,810551-050,2015/06/14,BLACK/DARK GREY-WHITE-CHROME,325200,0,0,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
2990,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,290000,852625-650,2016/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,286200,340000,180000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
2991,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,350000,AH7233-105,2018/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,234100,380000,150000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2993 entries, 0 to 2992
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   _id                 2993 non-null   object
 1   brand               2993 non-null   object
 2   prod_kr_name        2993 non-null   object
 3   rescent_price       2993 non-null   int64 
 4   model_no            2993 non-null   object
 5   release_date        2993 non-null   object
 6   colors              2993 non-null   object
 7   release_price       2993 non-null   int64 
 8   instant_buy_price   2993 non-null   int64 
 9   instant_sell_price  2993 non-null   int64 
 10  img_url             2993 non-null   object
dtypes: int64(4), object(7)
memory usage: 280.6+ KB


In [20]:
# 출시일자 날짜형식으로 전처리
df2['release_date'] = pd.to_datetime(df2['release_date'], format='%Y/%m/%d')
df2

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,200000,553558-053,1900-01-01,GREY/WHITE/CREAM,129000,202000,201000,https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,1555000,DM7866-140,2021-08-13,SAIL/BLACK/MILITARY BLUE,189000,1540000,2230000,https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,702000,555088-105,2020-11-13,SAIL/DARK MOCHA-BLACK-BLACK,199000,600000,818000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,312000,DH6927-111,2022-05-23,WHITE/BLACK/NEUTRAL GREY,249000,299000,313000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,244000,555088-108,2022-05-14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,209000,207000,270000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2988,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),0,CD7071-001,2019-05-25,BLACK/BLACK-WHITE-VARSITY RED,234600,0,0,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
2989,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,0,810551-050,2015-06-14,BLACK/DARK GREY-WHITE-CHROME,325200,0,0,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
2990,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,290000,852625-650,2016-12-18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,286200,340000,180000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
2991,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,350000,AH7233-105,2018-02-24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,234100,380000,150000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2993 entries, 0 to 2992
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   _id                 2993 non-null   object        
 1   brand               2993 non-null   object        
 2   prod_kr_name        2993 non-null   object        
 3   rescent_price       2993 non-null   int64         
 4   model_no            2993 non-null   object        
 5   release_date        2993 non-null   datetime64[ns]
 6   colors              2993 non-null   object        
 7   release_price       2993 non-null   int64         
 8   instant_buy_price   2993 non-null   int64         
 9   instant_sell_price  2993 non-null   int64         
 10  img_url             2993 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 280.6+ KB


In [22]:
# mysql import 하려면 cp949로 바꿔야한다.
# df2.to_csv('kream_detail_pro.csv', index=False, encoding='cp949')

In [24]:
df2.to_csv('C:/Users/psk/3rd_backup/kream_detail_pro8.csv', index=False)